In [1]:
# pip install scipy

In [2]:
%load_ext autoreload
%autoreload 2
import torch
from utils import BBandFChead, train_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/me.docker/.conda/envs/torch-p37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
# resnet = resnet50(pretrained=True, ) 
resnet.fc = nn.Identity() # removing pretrained last layer
resnet_linear = BBandFChead(resnet, hidden_dim=0, output_dim=101) # creating a model with frozen bb and only a linear eval head.

/home/me.docker/.conda/envs/torch-p37/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/.conda/envs/torch-p37/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from torch.utils.data import DataLoader
from caltech101 import load_caltech101

train_dataset, test_dataset = load_caltech101(data_path="/home/me.docker/work/data")
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1)


6941 1736


In [5]:
import torch.optim as optim
from utils import train_epoch

optimizer = optim.Adam(resnet_linear.parameters(), lr=0.001)
train_epoch(resnet_linear, train_dataloader, optimizer=optimizer, 
            max_samples=100, log_interval=10, epoch=0, silent=False)

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid 1593) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.

In [ ]:
train_model(resnet_linear, train_dataset=train_dataset, test_dataset=test_dataset, batch_size=16,
            lr=0.0001, epochs=20, samples_per_epoch=1000)

In [ ]:
from matplotlib import pyplot as plt
f, axes = plt.subplots(3,3, figsize=(15,15))
mult = len(train_dataset)//9
for i, ax in enumerate(axes.flatten()):
    ax.imshow(train_dataset[i*mult][0].permute(1,2,0))
    ax.title.set_text(train_dataset[i*mult][1])